In [1]:
from poke_env.environment.move import Move

from typing import Optional
import json

In [2]:
with open("../../pkmn_data/gen1randbats_dmg_calc.json", "r") as f:
    dmg_calc_results = json.load(f)

In [14]:
Move("hyperbeam", 1).category

<MoveCategory.PHYSICAL: 1>

In [6]:
from poke_env.environment.pokemon_type import PokemonType

len(PokemonType)

19

In [10]:
from poke_env.environment.pokemon import Pokemon

Pokemon(species="charizard", gen=1).type_1

<PokemonType.FIRE: 7>

In [70]:
def embed_move(attacker: Optional[str] = None, defender: Optional[str] = None, move_str: Optional[str] = None, dmg_calc = None, fainted=False):
    if fainted:
        return [0] * 27
    
    if defender not in dmg_calc[attacker] or move_str not in dmg_calc[attacker][defender]:
        dmg = [0, 0]
        recoil = [0, 0]
        recovery = [0, 0]
    else:
        dmg = dmg_calc[attacker][defender][move_str].get("damage", [0, 0])
        recoil = dmg_calc[attacker][defender][move_str].get("recoil", [0, 0])
        recovery = dmg_calc[attacker][defender][move_str].get("recovery", [0, 0])
        
    recovery_moves = {"recover": 0.5, "softboiled": 0.5, "rest": 1}
    if move_str in recovery_moves:
        recovery = [recovery_moves[move_str], recovery_moves[move_str]]
    
    move = Move(move_str, gen=1)
    
    accuracy = move.accuracy
    crit_ratio = move.crit_ratio
    priority = move.priority
    
    move_info = [accuracy, crit_ratio, priority]
    
    move_categories = ['physical', 'special', 'status']
    move_category = [0] * len(move_categories)
    move_category[move_categories.index(str(move.category.name).lower())] = 1
    
    boosts_order = ['atk', 'spa', 'def', 'spd', 'spe', 'accuracy']
    boosts = [0] * len(boosts_order)
    if move.boosts:
        for stat, boost in move.boosts.items():
            boosts[boosts_order.index(stat)] = boost

    status_order = ['par', 'slp', 'frz', 'brn', 'psn']
    status = [0] * len(status_order)
    if move.status:
        status[status_order.index(str(move.status.name).lower())] = 1
        
    volatile_statuses = ['confusion', 'substitute', 'reflect', 'flinch']
    volatile_status = [0] * len(volatile_statuses)
    if move.volatile_status:
        volatile_status[volatile_statuses.index(str(move.volatile_status).lower())] = 1
        
    for secondary in move.secondary:
        chance = float(secondary["chance"]) / 100
        if "boosts" in secondary:
            for stat, boost in secondary["boosts"].items():
                boosts[boosts_order.index(stat)] = boost * chance
        if "status" in secondary:
            status[status_order.index(str(secondary["status"]).lower())] = chance
        if "volatileStatus" in secondary:
            volatile_status[volatile_statuses.index(str(secondary["volatileStatus"]).lower())] = chance
    
    unique_moves = ["hyperbeam", "skyattack", "rest", "counter", "mimic"]
    unique_move = [0] * len(unique_moves)
    if move_str in unique_moves:
        unique_move[unique_moves.index(move_str)] = 1
    
    # print(dmg, recoil, recovery)
    # print(move_category, move_info)
    # print(boosts, status, volatile_status)
    # print(unique_move)
    
    return dmg + recoil + recovery + move_category + move_info + boosts + status + volatile_status

In [68]:
len(embed_move("Exeggutor", "Snorlax", "megadrain", dmg_calc_results))

27

In [69]:
TEST_MOVES = ["agility", "explosion", "megadrain", "hyperbeam", "substitute", 
              "thunderwave", "fireblast", "confuseray", "recover", "rest",
              "quickattack"]
for move in TEST_MOVES:
    # print(move)
    # print(embed_move("Exeggutor", "Snorlax", move, dmg_calc_results))
    embed_move("Exeggutor", "?", move, dmg_calc_results)

In [64]:
with open("../../pkmn_data/all_moves.txt", "r") as f:
    all_moves = f.read().splitlines()
    
clean_move = lambda move: move.replace(" ", "").replace("-", "").lower()


In [65]:
for move in all_moves:
    # print(move)
    embed_move("Exeggutor", "Snorlax", clean_move(move), dmg_calc_results)